### CS4423 - Networks
Angela Carnevale  
School of Mathematical and Statistical Sciences  
University of Galway

#### 3. Centrality Measures

# Week 5, lecture 2: More on Eigenvector Centrality. 

# Closeness and Betweenness  Centrality

In [ ]:
import networkx as nx
import pandas as pd
import yaml
from queue import Queue
opts = { "with_labels": True, "node_color": 'y'}

## Implementation and an example

We'll look at a recurring example while discussing the various centrality measures. Consider the following historical network of **florentine families** (from the 13th and 14th centuries), 
linked by **marital ties**.

Start by importing the necessary
`python` libraries into this jupyter notebook. In the following, we will make explicit use of `Pandas`, `numpy` and `yaml`.

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import yaml
opts = { "with_labels": True, "node_color": 'y'}

In [ ]:
G = nx.florentine_families_graph()
list(G.nodes())

**Note.** This version of the graph misses the isolated (= degree 0) node `'Pucci'` of the original graph [cf. Latora et al, Chapter 2]. We'll add it by hand. 

In [ ]:
G.add_node('Pucci')

In [ ]:
nx.draw(G, **opts)

A reason to look at this example is that in this context, we have further known indicators of the importance of these families (nodes) that we can compare with the node degree (and other centrality measures) in the graph `G`. These were the **wealth**, and the number of seats
on the city council (**priorates**) of each family.

In [ ]:
# this dictionary contains data on wealth

wealth = {
  'Acciaiuoli': 10, 'Albizzi': 36, 'Barbadori': 55, 'Bischeri': 44,
  'Castellani': 20, 'Ginori': 32, 'Guadagni': 8, 'Lamberteschi': 42,
  'Medici': 103, 'Pazzi': 48, 'Peruzzi': 49, 'Pucci': 3,
  'Ridolfi': 27, 'Salviati': 10, 'Strozzi': 146, 'Tornabuoni': 48,
}

# this dictionary contains data on political influence

priorates = {
  'Acciaiuoli': 53, 'Albizzi': 65, 'Barbadori': 'n/a', 'Bischeri': 12,
  'Castellani': 22, 'Ginori': 'n/a', 'Guadagni': 21, 'Lamberteschi': 0,
  'Medici': 53, 'Pazzi': 'n/a', 'Peruzzi': 42, 'Pucci': 0,
  'Ridolfi': 38, 'Salviati': 35, 'Strozzi': 74, 'Tornabuoni': 'n/a',
}

In [ ]:
nx.set_node_attributes(G, wealth, 'wealth')
nx.set_node_attributes(G, priorates, 'priorates')
nx.set_node_attributes(G, dict(G.degree()), 'degree')

In [ ]:
print(dict(G.degree()))

In [ ]:
G.nodes['Pazzi']

In [ ]:
dict(G.nodes(data=True))

In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

In [ ]:
nx.set_node_attributes(G, nx.degree_centrality(G), '$C_i^D$')

In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

* As the table shows, a high degree is often positively correlated with the other empirical indices of importance (wealth, political influence). 
* However, there are exceptions to that (e.g. the Guadagni family).
* This tells us that other centrality measures might be better suited in order to capture the importance of a node in a network (also depending on the nature of the network itself).



### Eigenvector centrality

As we need an irreducible matrix, we will remove the node `'Pucci'` before computing the eigenvector centrality.

In [ ]:
G.remove_node('Pucci')

The `networkx` package computes the eigenvector centraility of the nodes of a graph as follows:

In [ ]:
eigen_cen = nx.eigenvector_centrality(G)
eigen_cen

Let's attach the eigenvector centralities as node attributes
and display the resulting table. 

In [ ]:
eigen_cen = nx.eigenvector_centrality(G)
nx.set_node_attributes(G, eigen_cen, '$C_i^E$')

In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

Save the graph and its attributes for future use.

In [ ]:
with open('data/florentine.yml','w') as f:
    yaml.dump(G,f)

## Closeness Centrality

A node $x$ in a network can be regarded as being central, if it is **close** to (many) other nodes, 
as it can then quickly interact with them. 

As "lower distance &harr; greater importance", a simple way to measure closeness in this sense
is based on the sum of all the distances to the other nodes, as follows.

**Definition (Closeness Centrality).**
In a simple, **connected** graph $G$ of order $n$, the **closeness centrality** $c_i^C$ of node $i$
is defined as
$$
c_i^C = \frac{1}{\sum_{j=1}^n d_{ij}}.
$$

As always, together with the raw numbers, we consider the normalised ones.

The **normalised closeness centrality** of node $i$, defined as
$$
C_i^C = (n-1) c_i^C
$$
takes values in the interval $[0, 1]$.
</div>

**Problem.** How to compute $c_i^C$? 

**BFS again.**  

* The following `python` function implements
BFS for shortest distance from a previous lecture.  
* It takes a graph $G = (X, E)$ and a vertex $x \in X$
as its arguments. 
* It returns a **dictionary**, which assigns to each node its distance to $x$.

In [ ]:
def distances(G, x):
    
    # 1. init: set up the dictionary and a queue
    dists = { y: None for y in G } # distances
    Q = Queue() # queue of nodes to be visited
    dists[x] = 0
    Q.put(x)
    
    # 2. loop
    while not Q.empty(): 
        y = Q.get() 
        for z in G.neighbors(y):
            if dists[z] is None:
                dists[z] = dists[y] + 1
                Q.put(z)
    
    # 3. stop here
    return dists

* For example, the distance from node `'Medici'` to all nodes in `G`:

In [ ]:
d = distances(G, 'Medici')
print(d)
G.order()

**Note.** 

* On the isolated node `'Pucci'` the above would have returned an undefined distance from all nodes other than itself. 

* When a node is isolated, the sum of all the distances from it is $0$, causing a **division-by-zero error**. But we've removed the only isolated node in our graph so we're safe.

* Use `distances` to compute the normalized closeness centrality according to the above
definition.

In [ ]:
n = G.order()
closeness = { x : (n-1)/sum(distances(G, x).values()) for x in G }

In [ ]:
closeness

* Compare the results to the `networkx` version of closeness:

In [ ]:
nx.closeness_centrality(G)

In [ ]:
nx.closeness_centrality(G) == closeness

* Let's add those measurements to the table.

In [ ]:
nx.set_node_attributes(G, closeness, '$C_i^C$')

In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

We update our file with the most recent data for later use.

In [ ]:
with open('data/florentine.yml', 'w') as f:
    yaml.dump(G,f)
    

## Betweenness Centrality

When interactions between non-adjacent agents in a network depend
on middle actors (sitting on shortest paths between these agents), **power comes
to those in the middle**.  Betweenness centrality measures centrality
in terms of the number of shortest paths a node lies on.

**Definition (Betweenness Centrality).**
In a simple, connected graph $G$, the **betweenness centrality** $c_i^B$ of node $i$
is defined as
$$
c_i^B = \sum_{j \neq i} \sum_{k \neq i} \frac{n_{jk}(i)}{n_{jk}},
$$
where $n_{jk}$ denotes the **number** of shortest paths from
node $j$ to node $k$, and where $n_{jk}(i)$ denotes the
number of those shortest paths **passing through** node $i$.

That is, the betweenness centrality of node $i$ is the sum of the **proportions of shortest paths passing through $i$** among all shortest paths between any two vertices.

Once more, we'd like to get a normalised version of this measure, so as to make it comparable to the other measures we found.

The **normalised betweenness centrality** of node $i$, defined as
$$
C_i^B = \frac{c_i^B}{(n-1)(n-2)}
$$
takes values in the interval $[0, 1]$.
</div>

* Note that $(n-1)(n-2)$ is the largest number of shortest paths beween pairs of distinct nodes that a given node could possibly sit on.

* How to compute $C_i^B$?

**BFS once more.**  This time as a python function, which returns a **dictionary** that contains, for each node $x$, a list of **immediate predecessors** of $y$
in a shortest path from $x$ to $y$.  We have already seen that this another piece of information that BFS can determine
on the fly: when constructing a **spanning tree** while traversing a graph, we need to remember **one**
immediate predecessor for each newly discovered node.  Here we determine and remember **all** immediate
predecessors, requiring little if no extra work.

From this list of predecessors, one can then recursively reconstruct **all shortest paths** from $x$ to $y$.
We still need to keep track of the shortest path lengths in order to decide which neighbor $x$
actually is a predecessor of $y$.

In [ ]:
def predecessors(G, x):
    
    # 1. init: set up the two dictionaries and queue
    dists = { y: None for y in G } # distances
    preds = { y: [] for y in G } 
    Q = Queue()
    dists[x] = 0
    Q.put(x)
    
    # 2. loop
    while not Q.empty():
        y = Q.get()
        for z in G.neighbors(y):
            if dists[z] is None:
                dists[z] = dists[y] + 1
                preds[z].append(y)
                Q.put(z)
            elif dists[z] > dists[y]:
                preds[z].append(y)
    
    # 3. stop here
    return preds

In [ ]:
p = predecessors(G, 'Medici')
p

In [ ]:
nx.draw(G, **opts)

We'll see next week how these lists of predecessors can be used to form (and count) the shortest paths within $G$.

Using the **predecessor lists** with respect to $x$, the **shortest paths** from $x$ to $y$ can be enumerated recursively:
* if $y = x$: the shortest path from $x$ to itself is the empty path starting an ending at $x$.
* else, if $y \neq x$ then each shortest path from $x$ to $y$ travels through
  exactly one of $y$'s predecessors ... and ends in $y$.

So, in formulas, $S_x(x) = \{(x)\}$ and
$$
S_x(y) = \{ p + (y) : p \in S_x(z),\, z \in \mathrm{pre}_x(y)\}
$$

In [ ]:
def shortest_paths(x, y, pre):
    if x == y:
        return [[x]]
    paths = []
    for z in pre[y]:
        for path in shortest_paths(x, z, pre):
            paths.append(path + [y])
    return paths

In [ ]:
def spaths(x, y, pre):
    if x == y:
        return [[x]]
    return [ p + [y] for p in spaths(x, z, pre) for z in pre[y] ]

In [ ]:
shortest_paths('Medici', 'Bischeri', p)

* Now compute betweenness:

In [ ]:
betweenness = { x : 0.0 for x in G }

In [ ]:
for x in G:
    pre = predecessors(G, x)
    for y in G:
        paths = shortest_paths(x, y, pre)
        njk = len(paths)*(n-1)*(n-2)  # normalize
        for p in paths:
            for z in p[1:-1]:  # exclude endpoints
                betweenness[z] += 1/njk

In [ ]:
betweenness

* Compare the results to the `networkx` version of betweenness:

In [ ]:
nx.betweenness_centrality(G)

In [ ]:
nx.draw(G, **opts)

* Finally, let's add the normalized betweenness centralities as attributes to the
nodes of the graph, and display the resulting table.

In [ ]:
nx.set_node_attributes(G, betweenness, '$C_i^B$')

In [ ]:
pd.DataFrame.from_dict(
    dict(G.nodes(data=True)), 
    orient='index'
).sort_values('degree', ascending=False)

In [ ]:
with open('data/florentine.yml', 'w') as f:
    yaml.dump(G,f)
    

##  Summary

There are many different ways to be important.  As a node in a network, you are important if
* you have **many friends** (degree centrality)
* you have **important friends** (eigenvector centrality)
* you are **close** to many (closeness centrality)
* many interactions **pass through** you (betweenness centralty).

##  Code Corner

### `networkx`

* `closeness_centrality`: [[doc]](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.closeness_centrality.html#networkx.algorithms.centrality.closeness_centrality)
   
    
* `betweenness_centrality`: [[doc]](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html#networkx.algorithms.centrality.betweenness_centrality)

## Exercises

1. Recall that $C_i^C$ is the normalized closeness centrality of node $i$.  Why
   is $0 \leq C_i^C \leq 1$?  When is $C_i^C = 1$?  Is $C_i^C$ ever $0$?

2. Recall that $C_i^B$ is the normalized betweenness centrality of node $i$.
   Why is $0 \leq C_i^B \leq 1$?  When is $C_i^B = 1$?  Is $C_i^B$ ever $0$?
   
3. Determine the closeness centrality and the betweenness centrality of the nodes in some
   random trees.  What do you observe?
   
3. Compute the closeness centrality and the betweenness centrality of the nodes of the Petersen graph.
   What do you observe?